In [ ]:
model.to(device)

# для логов
train_losses, val_losses, val_rmse = [], [], []

for epoch in range(n_epochs):
    # === обучение ===
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)

    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    # === валидация ===
    model.eval()
    val_loss, val_preds, val_targets = 0.0, [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb)
            loss = criterion(preds, yb)

            val_loss += loss.item() * xb.size(0)
            val_preds.append(preds.detach().cpu())
            val_targets.append(yb.detach().cpu())

    epoch_val_loss = val_loss / len(val_loader.dataset)
    val_losses.append(epoch_val_loss)

    y_true = torch.cat(val_targets).numpy()
    y_pred = torch.cat(val_preds).numpy()
    rmse = np.sqrt(((y_true - y_pred) ** 2).mean())
    val_rmse.append(rmse)

    print(f"Epoch {epoch+1:2d}/{n_epochs} | "
          f"Train Loss: {epoch_train_loss:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f} | "
          f"Val RMSE: {rmse:.4f}")

# === графики ===
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Кривые обучения")
plt.legend()

plt.subplot(1,2,2)
plt.plot(val_rmse, marker="o", label="Val RMSE")
plt.xlabel("Epoch")
plt.ylabel("RMSE")
plt.title("Динамика RMSE")
plt.legend()
plt.show()

logs = {
    "train_losses": train_losses,
    "val_losses": val_losses,
    "val_rmse": val_rmse
}
return model, logs